# 01 - Adquisición de Datos: Dólar Observado BCCh

Este notebook descarga y procesa datos históricos del tipo de cambio USD/CLP desde el Banco Central de Chile.

## Objetivos
- Descargar 30 años de datos históricos (1995-2025)
- Procesar y guardar en formato CSV
- Verificar calidad de datos
- Generar resumen estadístico inicial

## 1. Setup y Configuración

In [ ]:
import os
import sys

# Agregar src al path para importar módulos
sys.path.append(os.path.abspath('..'))

from src.data.fetch_bcch import fetch_dolar_observado
from src.data.preprocess import process_and_save

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print(" Librerías importadas correctamente")

In [ ]:
# Configurar credenciales (asegúrate de tener las variables de entorno configuradas)
# export BC_USER="tu_email@ejemplo.com"
# export BC_PASS="tu_contraseña"

# Verificar que estén configuradas
bc_user = os.environ.get('BC_USER')
bc_pass = os.environ.get('BC_PASS')

if bc_user and bc_pass:
    print(f" Credenciales configuradas para: {bc_user}")
else:
    print(" ERROR: Credenciales no configuradas")
    print("Por favor configura las variables de entorno BC_USER y BC_PASS")

## 2. Descarga de Datos

In [ ]:
# Descargar datos de los últimos 30 años
print("Descargando datos del Banco Central de Chile...")
print("Serie: F073.TCO.PRE.Z.D (Dólar observado diario)")
print("Período: Últimos 30 años\n")

data = fetch_dolar_observado(years_back=30, verbose=True)

## 3. Procesamiento y Guardado

In [ ]:
# Procesar JSON -> DataFrame -> CSV
print("\nProcesando datos...")
df = process_and_save(data, verbose=True)

## 4. Inspección Inicial de Datos

In [ ]:
# Información general del DataFrame
print("\n" + "="*60)
print("INFORMACIÓN DEL DATASET")
print("="*60)
df.info()

In [ ]:
# Estadísticas descriptivas
print("\n" + "="*60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("="*60)
df.describe()

In [ ]:
# Análisis de valores faltantes
print("\n" + "="*60)
print("ANÁLISIS DE VALORES FALTANTES")
print("="*60)

total_rows = len(df)
missing_values = df['Valor'].isna().sum()
valid_values = df['Valor'].notna().sum()

print(f"Total de registros: {total_rows:,}")
print(f"Valores válidos: {valid_values:,} ({valid_values/total_rows*100:.2f}%)")
print(f"Valores faltantes (NaN): {missing_values:,} ({missing_values/total_rows*100:.2f}%)")

if 'statusCode' in df.columns:
    status_counts = df['statusCode'].value_counts()
    print("\nDistribución por statusCode:")
    print(status_counts)

## 5. Visualización Inicial

In [ ]:
# Gráfico de serie temporal completa
plt.figure(figsize=(16, 6))
plt.plot(df.index, df['Valor'], linewidth=0.8, alpha=0.8)
plt.title('Tipo de Cambio USD/CLP (1995-2025)', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor CLP', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nValor mínimo histórico: ${df['Valor'].min():.2f} CLP")
print(f"Valor máximo histórico: ${df['Valor'].max():.2f} CLP")
print(f"Valor actual (último): ${df['Valor'].iloc[-1]:.2f} CLP")

In [ ]:
# Distribución de valores
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['Valor'].dropna(), bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Valores USD/CLP', fontsize=14, fontweight='bold')
plt.xlabel('Valor CLP', fontsize=11)
plt.ylabel('Frecuencia', fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df['Valor'].dropna(), vert=True)
plt.title('Boxplot USD/CLP', fontsize=14, fontweight='bold')
plt.ylabel('Valor CLP', fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Resumen Final

In [ ]:
print("\n" + "="*60)
print("RESUMEN DE DESCARGA")
print("="*60)

print(f"\n Datos descargados exitosamente")
print(f" Archivo guardado en: data/raw/dolar_bcch.csv")
print(f" Total de observaciones: {len(df):,}")
print(f" Rango de fechas: {df.index.min().date()} a {df.index.max().date()}")
print(f" Tamaño del archivo: {os.path.getsize('../data/raw/dolar_bcch.csv') / 1024:.2f} KB")

print("\n Listo para continuar con el EDA (Análisis Exploratorio)")
print("Siguiente notebook: 02_eda.ipynb")